<a href="https://colab.research.google.com/github/AngelTroncoso/ADK/blob/main/ADK_Learning_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 ¡Bienvenido a tu Aventura ADK - Herramientas y Memoria! 🚀

¡Bienvenido, Agente Arquitecto! Este cuaderno es tu guía para otorgar a tus agentes de IA dos superpoderes esenciales: herramientas personalizadas y memoria conversacional.Al final de esta aventura, serás capaz de:- **Construir un Agente Fundamental**: Crear un agente de IA simple pero efectivo desde cero usando el Kit de Desarrollo de Agentes de Google (ADK).
- **Otorgar Nuevas Habilidades con Herramientas Personalizadas**: Enseñar a un agente a realizar nuevas tareas conectándolo a APIs externas, como un servicio de clima en tiempo real.
- **Crear un Equipo de Agentes**: Formar un sistema multiagente donde un agente principal pueda delegar tareas especializadas a otros agentes.
- **Dominar la Memoria Conversacional**: Comprender el papel crítico de las Sesiones para permitir que los agentes recuerden interacciones previas, gestionen retroalimentación y mantengan una conversación coherente.¡Vamos a comenzar esta aventura!

Hola, soy Qingyue (Annie) Wang, una defensora de desarrolladores e ingeniera de IA en **Google**, apasionada por ayudar a los desarrolladores a crear con tecnologías de IA y en la nube :)Si tienes preguntas sobre este cuaderno, contáctame en [LinkedIn](https://www.linkedin.com/in/qingyuewang/), [X](https://twitter.com/qingyuewang) o por correo electrónico anniewangtech0510@Gmail.com


```
  (\__/)
  (•ㅅ•)
  /づ  📚      Enjoy learning AI Agents :)
```


-------------
### 🎁 🛑 Requisito importante: ¡Configura tu entorno! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Get Your API Key HERE**: https://codelabs.developers.google.com/onramp/instructions#1

 -----------------------------------------------------------------------------

```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )             
```


## Parte 0: Configuración y Autenticación 🔑

Primero lo primero, preparemos todas nuestras herramientas. Este paso instala las bibliotecas necesarias y configura de manera segura tu clave de API de Google para que tus agentes puedan acceder al poder de Gemini.

In [ ]:
!pip install google-adk google-generativeai -q

# --- Importa todas las bibliotecas necesarias para toda nuestra aventura ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ ¡Todas las bibliotecas están listas para usar!")

In [ ]:
# --- Configura tu clave API de forma segura ---

# Solicite al usuario su clave de API de forma segura
api_key = getpass('Enter your Google API Key: ')

# Obtén tu clave API AQUÍ 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configura la biblioteca de IA generativa con la clave proporcionada
genai.configure(api_key=api_key)

# Establece la clave API como una variable de entorno para que ADK la use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

---
## Parte 1: Tu Primer Agente - El Genio de las Excursiones de un Día 🧞

¡Conoce tu primera creación! El `day_trip_agent` es un asistente simple pero poderoso. Lo estamos haciendo un poco más inteligente enseñándole a comprender **las restricciones de presupuesto**.

* **Agente**: El cerebro de la operación, definido por sus instrucciones, herramientas y el modelo de IA que utiliza.

* **Sesión**: El historial de la conversación. Para este agente simple, es solo un contenedor para una única solicitud-respuesta.

* **Ejecutor**: El motor que conecta al `Agente` y la `Sesión` para procesar tu solicitud y obtener una respuesta.

```
+--------------------------------------------------+
|         Agente de Excursiones Espontáneas 🤖     |
|--------------------------------------------------|
|  Model: gemini-2.5-flash                         |
|  Description:                                    |
|   Generates full-day trip itineraries based on   |
|   mood, interests, and budget                    |
|--------------------------------------------------|
|  🔧 Tools:                                       |
|   - Google Search                                |
|--------------------------------------------------|
|  🧠 Capabilities:                                |
|   - Budget Awareness (cheap / splurge)           |
|   - Mood Matching (adventurous, relaxing, etc.)  |
|   - Real-Time Info (hours, events)               |
|   - Morning / Afternoon / Evening plan           |
+--------------------------------------------------+

            ▲
            |
    +------------------+
    |   User Input     |
    |------------------|
    |  Mood            |
    |  Interests       |
    |  Budget          |
    +------------------+

            |
            ▼

+--------------------------------------------------+
|             Output: Markdown Itinerary           |
|--------------------------------------------------|
| - Time blocks (Morning / Afternoon / Evening)    |
| - Venue names with links and hours               |
| - Budget-matching activities                     |
+--------------------------------------------------+
```


In [ ]:
# --- Agent Definition ---

def create_day_trip_agent():
    """Crear el agente Generador de Excursiones Espontáneas"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agente especializado en generar itinerarios completos de un día de manera espontánea, según el estado de ánimo, intereses y presupuesto.",
        instruction="""
        Eres el Generador de "Excursiones de un Día Espontáneas" 🚗 - un asistente de IA especializado que crea itinerarios completos y atractivos para todo el día.

        Tu Misión::
        Transforma un estado de ánimo o interés simple en una aventura de excursión de un día completa con detalles en tiempo real, respetando un presupuesto.

        Directrices:
        1. **Consciente del presupuesto**: Presta mucha atención a las indicaciones sobre el presupuesto como 'barato', 'asequible' o 'lujo'. Utiliza Google Search para encontrar actividades (museos gratuitos, parques, atracciones de pago) que se ajusten al presupuesto del usuario.
        2. **Estructura de día completo**: Crea actividades para la mañana, la tarde y la noche.
        3. **Enfoque en tiempo real**: Busca los horarios de funcionamiento actuales y eventos especiales.
        4. **Coincidencia con el estado de ánimo**: Alinea las sugerencias con el estado de ánimo solicitado (aventurero, relajante, artístico, etc.).
        DEVOLVER el itinerario en FORMATO MARKDOWN con bloques de tiempo claros y nombres específicos de lugares.
        """,
        tools=[google_search]
    )

day_trip_agent = create_day_trip_agent()
print(f"🧞 Agent '{day_trip_agent.name}' está creado y listo para la aventura!")

In [ ]:
# --- Una función auxiliar para ejecutar nuestros agentes ---
# Usaremos esta función a lo largo del cuaderno para facilitar la ejecución de consultas

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Inicializa un ejecutor y ejecuta una consulta para un agente y sesión determinados."""
    print(f"\n🚀 Ejecutando consulta para el agente: '{agent.name}' en sesión: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Inicializar nuestro Servicio de Sesión ---
# Este único servicio gestionará todas las diferentes sesiones en nuestro cuaderno.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [ ]:
# --- ¡Vamos a probar el Genio de Excursiones de un Día! ---

async def run_day_trip_genie():
    # Crear una nueva sesión de un solo uso para esta consulta
    day_trip_session = await session_service.create_session(
        app_name=day_trip_agent.name,
        user_id=my_user_id
    )

    # ¡Nota la nueva restricción presupuestaria en la consulta!
    query = "Plan a relaxing and artsy day trip near Sunnyvale, CA. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(day_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

---
## Parte 2: Potenciando Agentes con Herramientas Personalizadas 🛠️

Hasta ahora, hemos utilizado la poderosa herramienta incorporada `GoogleSearch`. Pero el verdadero poder de los agentes surge al conectarlos con tu propia lógica y fuentes de datos.

Aquí es donde entran las **herramientas personalizadas**. Exploraremos tres patrones para dotar a tu agente de nuevas habilidades, utilizando ejemplos prácticos del mundo real.

### 2.1 La `FunctionTool` Simple: Llamando a una API de Clima en Tiempo Real

La forma más directa de crear una herramienta es escribiendo una función en Python. Esto es perfecto para tareas sincrónicas como obtener datos de una API.

**Concepto clave:** El **docstring** de la función es fundamental. El ADK lo utiliza como la descripción oficial de la herramienta, que el LLM lee para comprender su propósito, sus parámetros y cuándo usarla.

En este ejemplo, crearemos una herramienta que llame a la **API gratuita y pública del Servicio Meteorológico Nacional de EE. UU.** para obtener un pronóstico en tiempo real. ¡No se necesita clave de API!

In [ ]:
# --- Definición de herramienta: Una función que llama a una API pública en vivo ---
import requests
import json

# Una búsqueda simple para evitar necesitar una API de geocodificación separada para este ejemplo
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')")

# Encuentra las coordenadas de la ubicación
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # La API de NWS requiere 2 pasos: 1. Obtener la URL del pronóstico a partir de las coordenadas.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Generar una excepción para códigos de estado incorrectos
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Obtén el pronóstico real desde la URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extraer los detalles relevantes del pronóstico
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Definición del Agente: Un agente que UTILIZA la nueva herramienta ---

weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

In [ ]:
# --- Probemos el Planificador Consciente del Clima ---

async def run_weather_planner_test():
    weather_session = await session_service.create_session(app_name=weather_agent.name, user_id=my_user_id)
    query = "I want to go hiking near Lake Tahoe, what's the weather like?"
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_planner_test()

## 2.2 El Agente como Herramienta: Consultando a un Especialista 🧑‍🍳

¿Por qué construir un agente que haga todo cuando puedes construir un **equipo de agentes especialistas?** El patrón de **Agente-como-Herramienta** permite que un agente delegue una tarea a otro agente.

**Concepto clave:** Esto es diferente de un subagente. Cuando el Agente A llama al Agente B como una herramienta, la respuesta del Agente B se pasa **de vuelta al Agente A**. Luego, el Agente A utiliza esa información para formar su propia respuesta final al usuario. Es una forma poderosa de componer comportamientos complejos a partir de agentes más simples, enfocados y reutilizables.

### Cómo funciona

Nuestro agente de nivel superior, el `trip_data_concierge_agent`, actúa como el **Orquestador**. Tiene dos herramientas a su disposición:

1.  `call_db_agent`: Una función que llama internamente a nuestro `db_agent` para obtener datos en bruto.
2.  `call_concierge_agent`: Una función que llama a la `concierge_agent`.

la `concierge_agent` tiene en sí mismo una herramienta: la `food_critic_agent`.

El flujo para una consulta compleja es:

1. **Usuario** solicita al `trip_data_concierge_agent` un hotel y un restaurante cercano.
2. El **Orquestador** primero llama a `call_db_agent` para obtener datos del hotel.
3. Los datos se guardan en `tool_context.state`.
4. Luego, el **Orquestador** llama a `call_concierge_agent`, que recupera los datos del hotel del contexto.
5. El `concierge_agent` recibe la solicitud y decide que necesita usar su propia herramienta, el `food_critic_agent`.
6. El `food_critic_agent` proporciona una recomendación ingeniosa.
7. El `concierge_agent` obtiene la respuesta del crítico y la formatea cortésmente.8. Esta respuesta final y pulida se devuelve al **Orquestador**, que la presenta al usuario.

                         +-----------------------------------------------------------+
                         |              🧭 Trip Data Concierge Agent                 |
                         |-----------------------------------------------------------|
                         |  Model: gemini-2.5-flash                                  |
                         |  Description:                                             |
                         |   Orchestrates database query and travel recommendation  |
                         |-----------------------------------------------------------|
                         |  🔧 Tools:                                                |
                         |   1. call_db_agent                                        |
                         |   2. call_concierge_agent                                 |
                         +-----------------------------------------------------------+
                                      /                                \
                                     /                                  \
                                    ▼                                    ▼
        +-------------------------------------------+    +---------------------------------------------+
        |            🔧 Tool: call_db_agent         |    |         🔧 Tool: call_concierge_agent        |
        |-------------------------------------------|    |---------------------------------------------|
        | Calls: db_agent                           |    | Calls: concierge_agent                       |
        |                                           |    | Uses data from db_agent for recommendations |
        +-------------------------------------------+    +---------------------------------------------+
                                |                                          |
                                ▼                                          ▼
       +--------------------------------------------+   +------------------------------------------------+
       |              📦 db_agent                   |   |             🤵 concierge_agent                  |
       |--------------------------------------------|   |------------------------------------------------|
       | Model: gemini-2.5-flash                    |   | Model: gemini-2.5-flash                         |
       | Role: Return mock JSON hotel data          |   | Role: Hotel staff that handles user Q&A        |
       +--------------------------------------------+   | Tools:                                          |
                                                         |  - food_critic_agent                           |
                                                         +------------------------------------------------+
                                                                                 |
                                                                                 ▼
                                                       +------------------------------------------------+
                                                       |          🍽️ food_critic_agent                  |
                                                       |------------------------------------------------|
                                                       | Model: gemini-2.5-flash                         |
                                                       | Role: Gives a witty restaurant recommendation   |
                                                       +------------------------------------------------+


In [ ]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Supongamos que 'db_agent' es un agente NL2SQL predefinido
# Para este ejemplo, crearemos agentes de marcador de posición.

db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}")

# --- 1. Definir los Agentes Especialistas ---

# El Crítico Gastronómico sigue siendo el especialista más profundo
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# El conserje sabe cómo usar al crítico gastronómico
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Definir las herramientas para el orquestador ---

async def call_db_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("--- TOOL CALL: call_db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )
    # Almacenar los datos recuperados en el estado del contexto
    tool_context.state["retrieved_data"] = db_agent_output
    return db_agent_output


async def call_concierge_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("--- TOOL CALL: call_concierge_agent ---")
    # Recuperar los datos obtenidos por la herramienta anterior
    input_data = tool_context.state.get("retrieved_data", "No data found.")

    # Formular un nuevo prompt para el conserje, proporcionándole el contexto de los datos
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Definir el Agente Orquestador de Nivel Superior ---

trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")

In [ ]:
# --- Probemos al Agente Conserje de Datos de Viaje -----

async def run_trip_data_concierge():
    """
    Sets up a session and runs a query against the top-level
    trip_data_concierge_agent.
    """
    # Crear una nueva sesión de un solo uso para esta consulta
    concierge_session = await session_service.create_session(
        app_name=trip_data_concierge_agent.name,
        user_id=my_user_id
    )

    # Esta consulta está diseñada específicamente para activar el proceso completo de dos pasos:
    # 1. Obtener datos del db_agent.
    # 2. Obtener una recomendación del concierge_agent basada en esos datos.
    query = "Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews."
    print(f"🗣️ User Query: '{query}'")

    # Llamamos a nuestra función de ayuda existente con el agente orquestador de nivel superior
    await run_agent_query(trip_data_concierge_agent, query, concierge_session, my_user_id)

# Ejecuta la prueba
await run_trip_data_concierge()

---
## Parte 3: Agente con Memoria - El Planificador Adaptativo 🗺️

Ahora, veamos un agente que no solo **recuerda**, sino que también **se adapta**. Desafiaremos al `multi_day_trip_agent` a replanificar parte de su itinerario basado en nuestros comentarios. Esta es una prueba mucho más realista de la IA conversacional.

```
+-----------------------------------------------------+
|         Adaptive Multi-Day Trip Agent 🗺️           |
|-----------------------------------------------------|
|  Model: gemini-2.5-flash                            |
|  Description:                                       |
|   Builds multi-day travel itineraries step-by-step, |
|   remembers previous days, adapts to feedback       |
|-----------------------------------------------------|
|  🔧 Tools:                                          |
|   - Google Search                                   |
|-----------------------------------------------------|
|  🧠 Capabilities:                                   |
|   - Memory of past conversation & preferences       |
|   - Progressive planning (1 day at a time)          |
|   - Adapts to user feedback                         |
|   - Ensures activity variety across days            |
+-----------------------------------------------------+

            ▲
            |
    +---------------------------+
    |     User Interaction      |
    |---------------------------|
    | - Destination             |
    | - Trip duration           |
    | - Interests & feedback    |
    +---------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Day-by-Day Itinerary Generation              |
|-----------------------------------------------------|
|  🗓️ Day N Output (Markdown format):                 |
|   - Morning / Afternoon / Evening activities        |
|   - Personalized & context-aware                    |
|   - Changes accepted, feedback acknowledged         |
+-----------------------------------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Next Day Planning Triggered 🚀               |
|-----------------------------------------------------|
| - Builds on prior days                              |
| - Avoids repetition                                 |
| - Asks user for confirmation before proceeding      |
+-----------------------------------------------------+
```


In [ ]:
# --- Definición del Agente: El Planificador Adaptativo ---

def create_multi_day_trip_agent():
    """Create the Progressive Multi-Day Trip Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback.",
        instruction="""
        You are the "Adaptive Trip Planner" 🗺️ - an AI assistant that builds multi-day travel itineraries step-by-step.

        Your Defining Feature:
        You have short-term memory. You MUST refer back to our conversation to understand the trip's context, what has already been planned, and the user's preferences. If the user asks for a change, you must adapt the plan while keeping the unchanged parts consistent.

        Your Mission:
        1.  **Initiate**: Start by asking for the destination, trip duration, and interests.
        2.  **Plan Progressively**: Plan ONLY ONE DAY at a time. After presenting a plan, ask for confirmation.
        3.  **Handle Feedback**: If a user dislikes a suggestion (e.g., "I don't like museums"), acknowledge their feedback, and provide a *new, alternative* suggestion for that time slot that still fits the overall theme.
        4.  **Maintain Context**: For each new day, ensure the activities are unique and build logically on the previous days. Do not suggest the same things repeatedly.
        5.  **Final Output**: Return each day's itinerary in MARKDOWN format.
        """,
        tools=[google_search]
    )

multi_day_agent = create_multi_day_trip_agent()
print(f"🗺️ Agent '{multi_day_agent.name}' is created and ready to plan and adapt!")

### Scenario 3a: Agent WITH Memory (Using a SINGLE Session) ✅

First, let's see the correct way to do it. We will use the **exact same `trip_session` object** for the entire conversation. Watch how the agent remembers the context from Turn 1 to correctly handle the requests in Turn 2 and 3.

In [ ]:
# --- Escenario 2: Prueba de Adaptación y Memoria ---

async def run_adaptive_memory_demonstration():
    print("### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###")

    # Crear UNA sesión que reutilizaremos para toda la conversación
    trip_session = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our trip: {trip_session.id}")

    # --- Turno 1: El usuario inicia el viaje ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, trip_session, my_user_id)

    # --- Turno 2: El usuario DA COMENTARIOS y solicita un CAMBIO ---
    # ¡Usamos el MISMO objeto `trip_session`!
    query2 = "That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?"
    print(f"\n🗣️ User (Turn 2 - Feedback): '{query2}'")
    await run_agent_query(multi_day_agent, query2, trip_session, my_user_id)

    # --- Turno 3: El usuario confirma y pide continuar ---!
    query3 = "Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind."
    print(f"\n🗣️ User (Turn 3 - Confirmation): '{query3}'")
    await run_agent_query(multi_day_agent, query3, trip_session, my_user_id)

await run_adaptive_memory_demonstration()

### Escenario 3b: Agente SIN Memoria (Usando sesiones SEPARADAS) ❌

Ahora, veamos qué pasa si arruinamos la gestión de nuestra sesión. Aquí, le daremos al agente un caso de amnesia creando una **sesión completamente nueva y separada para cada turno**.

Presta mucha atención a la respuesta del agente a la segunda consulta. ¡Como está en una nueva sesión, no tiene memoria del viaje a Lisboa que acabamos de discutir!

In [ ]:
# --- Escenario 2b: Demostrando FALLO DE MEMORIA ---

async def run_memory_failure_demonstration():
    print("\n" + "#"*60)
    print("### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###")
    print("#"*60)

    # --- Turno 1: El usuario inicia el viaje en la PRIMERA sesión ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    session_one = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a session for Turn 1: {session_one.id}")
    print(f"🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, session_one, my_user_id)

    # --- Turno 2: El usuario pide continuar... pero en una sesión completamente NUEVA ---
    query2 = "Yes, that looks perfect! Please plan Day 2."
    session_two = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a BRAND NEW session for Turn 2: {session_two.id}")
    print(f"🗣️ User (Turn 2): '{query2}'")
    await run_agent_query(multi_day_agent, query2, session_two, my_user_id)

await run_memory_failure_demonstration()

¿Ves? ¡El agente estaba confundido! Probablemente preguntó cuál era el destino o de qué viaje estábamos hablando. Como la segunda consulta se realizó en una sesión nueva y aislada, el agente no recordaba haber planificado el Día 1 en Lisboa.

¡Esto ilustra perfectamente por qué **gestionar las sesiones es la clave para construir agentes verdaderamente conversacionales!**

---
## 🎉 ¡Felicidades! 🎉

¡Felicidades por completar tu aventura en ADK sobre Herramientas y Memoria! Has dado un gran salto al pasar de construir agentes de un solo uso a crear sistemas de IA dinámicos y con estado.

Recapitulemos los conceptos poderosos que has dominado:

- **Agente Fundamental y Herramientas**: Comenzaste construyendo un "Genio de Excursiones de un Día" y lo equipaste con su primera herramienta, GoogleSearch.
- **Herramientas de Funciones Personalizadas**: Le diste a tu agente un nuevo sentido creando una herramienta personalizada para obtener datos en tiempo real de la API del Servicio Meteorológico Nacional de EE. UU.
- **Agente como Herramienta**: Orquestaste una jerarquía sofisticada donde los agentes delegan tareas a otros agentes más especializados, creando un equipo colaborativo.
- **El Poder de la Memoria**: Lo más importante es que experimentaste de primera mano cómo gestionar una sesión persistente permite a un agente recordar el contexto, adaptarse a la retroalimentación del usuario y llevar a cabo una conversación significativa de múltiples turnos.

```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```
